<img src="https://raw.githubusercontent.com/GeorgeTelles/georgetelles/f69531ec6b293b5148563588a764c010015d315e/logo_clara.png" width="300" align="left"/>

------
# **Algoritmo Automatizado de Trading com Python e MetaTrader5**
------

Este projeto implementa um algoritmo em Python que aplica uma estratégia de trading a múltiplos ativos financeiros simultaneamente, utilizando o MetaTrader5 para executar operações de compra automaticamente.

## Funcionalidades

1. **Instalação e Importação de Bibliotecas**
   - Utiliza a biblioteca `MetaTrader5` para comunicação com a plataforma de trading.
   - Importa outras bibliotecas necessárias para manipulação de dados e cálculos financeiros.

2. **Configuração Inicial**
   - Conecta ao MetaTrader5 e configura os ativos desejados para análise.
   - Verifica a disponibilidade dos ativos na plataforma.

3. **Criação do Screening de Ativos**
   - **Download de Dados**: Coleta dados históricos de cotações para cada ativo.
   - **Cálculo de Indicadores**: Adiciona Médias Móveis Simples (SMA) de 20 e 200 períodos aos dados dos ativos.
   - **Regras de Estratégia**: Define regras para identificar quais ativos atendem aos critérios da estratégia, como:
     - A SMA de 20 períodos deve ser maior que a SMA de 200 períodos.
     - O preço de fechamento deve estar acima da SMA de 20 períodos.

4. **Automatização de Trades**
   - Monitora os ativos periodicamente para verificar se atendem às condições da estratégia.
   - Executa automaticamente ordens de compra para os ativos que cumprem os critérios, utilizando o MetaTrader5.
   - Implementa controle para evitar múltiplas ordens para o mesmo ativo e ajusta o volume, stop loss e take profit das ordens.

## Requisitos

- MetaTrader5 instalado e configurado.
- Python 3.x.
- Bibliotecas: `MetaTrader5`, `pandas`, `numpy`, `pytz`, `mplfinance`, `plotly`.

Este algoritmo é ideal para traders que desejam automatizar a aplicação de estratégias de trading em múltiplos ativos, economizando tempo e potencialmente melhorando a eficiência das operações.


# 1. Instalando e Importando Bibliotecas

In [ ]:
%pip install MetaTrader5

In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
import pytz
from time import sleep

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import MetaTrader5 as mt5
import mplfinance as mpf

# 2. Iniciar o MT5 e configurar os ativos desejados

In [ ]:
mt5.initialize()

In [ ]:
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=5)
data_fim = datetime.now()

In [ ]:
lista_tickers = ["ABEV3", "ALPA4", "AZZA3", "ASAI3", "AZUL4", "B3SA3", "BBAS3",
                "BBDC3", "BBDC4", "BBSE3", "BEEF3", "BPAC11", "BPAN4", "BRAP4", "BRFS3",
                "BRKM5", "CASH3", "CCRO3", "CIEL3", "CMIG4", "CMIN3", "COGN3", "CPFE3",
                "CPLE6", "CRFB3", "CSAN3", "CSNA3", "CVCB3", "CYRE3",  "DXCO3", "ECOR3",
                "EGIE3", "ELET3", "ELET6", "EMBR3", "ENEV3", "ENGI11", "EQTL3",
                "EZTC3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "HAPV3", "HYPE3", "IGTI11",
                "ITSA4", "ITUB4", "JBSS3", "KLBN11", "LREN3", "LWSA3", "MGLU3", "MRFG3",
                "MRVE3", "MULT3", "NTCO3", "PCAR3", "PETR3", "PETR4", "PETZ3", "PRIO3",
                "QUAL3", "RADL3", "RAIL3", "RAIZ4", "RDOR3", "RENT3", "RRRP3", "SANB11",
                "SBSP3", "SLCE3", "SMTO3", "SUZB3", "TAEE11", "TIMS3", "TOTS3",
                "UGPA3", "USIM5", "VALE3", "VBBR3", "VIVT3", "WEGE3", "YDUQ3"]

**Verificando se os ativos existem**

In [ ]:
for ticker in lista_tickers:
    selected=mt5.symbol_select(ticker,True)
    if selected:
        print(ticker,"incluído")
    if not selected:
        print(ticker,"não localizado")

# 3. Criar screening de ativos

### 3.1. Baixar e organizar os dados de cotações

In [ ]:
cotacoes_mt5_all = []

for ticker in lista_tickers:
    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M5, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_ticker.drop("time", axis=1, inplace=True)
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)

In [ ]:
list(enumerate(lista_tickers))
pass

### 3.2. Adicionar indicadores e regras do screening

**Neste caso estou adiciando a Média Movel de 20 Periodos e a Média Movel de 200 Periodos**

In [ ]:
for i in range(0,len(cotacoes_mt5_all)):
  cotacoes_mt5_all[i]['SMA_20p'] = cotacoes_mt5_all[i]['close'].rolling(20).mean()
  cotacoes_mt5_all[i]['SMA_200p'] = cotacoes_mt5_all[i]['close'].rolling(200).mean()
  cotacoes_mt5_all[i] = cotacoes_mt5_all[i].dropna()

**2 regras simples:**

1) Ativos cuja **MM_20p seja maior que a MM_200p**
2) Ativos cujo **fechamento esteja acima da MM_20p**

In [ ]:
def screening_ativos(df):
    cond1 = df.SMA_20p.iloc[-1] > df.SMA_200p.iloc[-1]
    cond2 = df["open"].iloc[-1] > df["SMA_20p"].iloc[-1]

    return(cond1 & cond2)

**Mostra os ativos que atendem a condição**

In [ ]:
ls_screening_Regra_SMA = []

for i in range(0,len(cotacoes_mt5_all)):
    if (cotacoes_mt5_all[i].iloc[-1]['SMA_20p'] > cotacoes_mt5_all[i].iloc[-1]['SMA_200p']) & \
      (cotacoes_mt5_all[i].iloc[-1]['close'] > cotacoes_mt5_all[i].iloc[-1]['SMA_20p']):
        ls_screening_Regra_SMA.append((lista_tickers[i]))
        print("O ativo ("+lista_tickers[i]+") atende às regras da estrategia")
    else:
        # print(lista_tickers[i],"não atende")
        pass
print(f"{len(ls_screening_Regra_SMA)} Ativos atendem ás regras da estrategia")

# 4. Automatizando o trade baseado na estrategia

In [ ]:
timezone = pytz.timezone("America/Sao_Paulo")
data_inicio = datetime.now() - timedelta(days=3)
data_fim = datetime.now()

contagem = 0

while contagem < 15:

    print('------- Inicio do',contagem,' round -------')
    for ticker in lista_tickers:

        cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_M1, data_inicio, data_fim)
        cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
        cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
        cotacoes_mt5_ticker['ticker'] = str(ticker)
        cotacoes_mt5_ticker['SMA_rapida'] = cotacoes_mt5_ticker['close'].rolling(10).mean()
        cotacoes_mt5_ticker['SMA_lenta'] = cotacoes_mt5_ticker['close'].rolling(20).mean()

        if (cotacoes_mt5_ticker.iloc[-1]['SMA_rapida'] > cotacoes_mt5_ticker.iloc[-1]['SMA_lenta']) & \
            (cotacoes_mt5_ticker.iloc[-1]['close'] > cotacoes_mt5_ticker.iloc[-1]['SMA_rapida']):
            print("O ativo indexado como",ticker, "atende às regras e será comprado")

            positions = mt5.positions_get(symbol=ticker)
            if len(positions)==0:
                qtd = 100.0
                tick_min = mt5.symbol_info(ticker).point
                preco = mt5.symbol_info_tick(ticker).ask
                ordem_compra = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": ticker,
                    "volume": qtd,
                    "type": mt5.ORDER_TYPE_BUY,
                    "price": preco,
                    "sl": preco - 8 * tick_min,
                    "tp": preco + 8 * tick_min,
                    "magic": 1,
                    "comment": "Automatizando trades",
                    "type_time": mt5.ORDER_TIME_DAY,
                    "type_filling": mt5.ORDER_FILLING_RETURN,
                    }
                result_compra = mt5.order_send(ordem_compra)
            else:
                print(ticker,"já posicionado")

        else:
            # print(ticker,"não atende ao screening")
            pass

        sleep(1)

    contagem += 1
    sleep(30)